<a href="https://colab.research.google.com/github/Bolshartd/test_parsing/blob/main/parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import bs4
import pandas as pd


In [ ]:
# создаём запрос на источник
r = requests.get('https://matchtv.ru/football/england/stats')
r

<Response [200]>

In [ ]:
# запускаем парсер
soup = bs4.BeautifulSoup(r.text, 'html.parser')

In [ ]:
attributes = []
positions = []
# находим данные (часть 1)
for i in soup.find_all('td', {'class': "info table-link"}):
  for j in i.find_all('a'):
    club = j.get('href')
    # запускаем парсер на нужную страницу
    r1 = requests.get(f'https://matchtv.ru{club}/players')
    soup1 = bs4.BeautifulSoup(r1.text, 'html.parser')
    # находим нужные данные (имя, возраст, рост, вес)
    for s in soup1.find_all('tr', {'class': 'club-lineup__item'}):
      attributes.append(s.text)

In [ ]:
# создаем датафрейм по имеющимся данным (имя, возраст, рост, вес)
df_all = pd.DataFrame(attributes, columns=['date'])

In [ ]:
# обрабатываем данные
df_all = df_all.replace(to_replace='  ', value='', regex=True)
df_all = df_all.replace(to_replace='\n', value=',', regex=True)
df_all['date'] = df_all['date'].str.strip(',')
df_all = df_all.replace(to_replace=',,', value=',', regex=True)
new_df_all = df_all['date'].str.split(',', expand=True)
new_df_all.set_index(0, inplace=True)
new_df_all.drop('Имя', axis=0, inplace=True)

In [ ]:
final_df = new_df_all.reset_index()
total = final_df.fillna('-')
total.columns = [['Имя', 'Возраст', 'Рост', 'Вес']]

In [ ]:
nations = []
clubs = []
coachs = []
# находим данные (часть 2)
for i in soup.find_all('td', {'class': "info table-link"}):
  for j in i.find_all('a'):
    # запускаем парсер на нужную страницу
    club = j.get('href')
    r1 = requests.get(f'https://matchtv.ru{club}/players')
    soup1 = bs4.BeautifulSoup(r1.text, 'html.parser')
  # находим нужные данные (национальность, тренер, клуб)
  for s in soup1.find_all('img', {'class': 'club-lineup__icon'}):
    s = str(s)
    a = s.find('alt="')
    b = s.find('" cl')
    nations.append(s[a + 5:b])
    for club in soup1.find_all('h1', {'class': 'heading caption__heading'}):
      clubs.append(club.text)
    for coach in soup1.find_all('div', {'class': 'heading-5'}):
      coachs.append(coach.text)

In [ ]:
# создаем датафрейм по имеющимся данным (национальность, тренер, клуб)
df_n = pd.DataFrame([nations, coachs, clubs]).T
df_n.columns = [['Национальность', 'Тренер', 'Клуб']]

In [ ]:
# объединяем данные (получаем промежуточную таблицу)
table = total.join(df_n)
table

,Имя,Возраст,Рост,Вес,Национальность,Тренер,Клуб
0,Рунарссон Рунар,28,186,76,Iceland,Микель Артета,Арсенал
1,Рэмсдейл Арон,24,188,77,England,Микель Артета,Арсенал
2,Тернер Мэтт,28,191,80,United states,Микель Артета,Арсенал
3,Хейн Карл-Якоб,20,193,-,Estonia,Микель Артета,Арсенал
4,Хиллсон Джеймс,22,-,-,England,Микель Артета,Арсенал
...,...,...,...,...,...,...,...
774,Армстронг Адам,26,179,70,England,Рубен Селлес,Саутгемптон
775,Мара Секу,20,184,76,France,Рубен Селлес,Саутгемптон
776,Морган Джиммм,17,-,-,England,Рубен Селлес,Саутгемптон
777,Онуачу Пауль,28,201,84,Nigeria,Рубен Селлес,Саутгемптон


In [ ]:
positions = []
# находим данные (часть 3)
for i in soup.find_all('td', {'class': "info table-link"}):
  for j in i.find_all('a'):
    club = j.get('href')
    # запускаем парсер на нужную страницу
    r2 = requests.get(f'https://matchtv.ru{club}/players')
    soup2 = bs4.BeautifulSoup(r2.text, 'html.parser')
    # находим нужные данные (позиция)
    for position in soup2.find_all('section', {'class': 'content-section content-section_underlined_header content-section_no_padding content-section_margin_top'}):
        positions.append(position.text)

In [ ]:
# обрабатываем данные (позиция)
a =[]
for i, row in table['Имя'].iterrows():
    a.append(row['Имя'])
w = []
for j in positions:
    for h in a:
        if h in j:
            a1 = j.find("'\n")
            b1 = j.find('\nСостав')
            k = j[a1+7:b1-10]
            w.append(f'{h}, {k}')

In [ ]:
# создаем датафрейм по имеющимся данным (позиция)
df_p = pd.DataFrame(w)
df_p = df_p[0].str.split(',', expand=True)
df_p.columns = [['Имя', 'Позиция']]

In [ ]:
# объединяем все данные в итоговую позицию
df_final = pd.merge(table, df_p)
df_final.drop_duplicates(inplace=True)
finalTable = df_final.reset_index(drop=True)
finalTable

,Имя,Возраст,Рост,Вес,Национальность,Тренер,Клуб,Позиция
0,Рунарссон Рунар,28,186,76,Iceland,Микель Артета,Арсенал,Вратари
1,Рэмсдейл Арон,24,188,77,England,Микель Артета,Арсенал,Вратари
2,Тернер Мэтт,28,191,80,United states,Микель Артета,Арсенал,Вратари
3,Хейн Карл-Якоб,20,193,-,Estonia,Микель Артета,Арсенал,Вратари
4,Хиллсон Джеймс,22,-,-,England,Микель Артета,Арсенал,Вратари
...,...,...,...,...,...,...,...,...
774,Армстронг Адам,26,179,70,England,Рубен Селлес,Саутгемптон,Нападающие
775,Мара Секу,20,184,76,France,Рубен Селлес,Саутгемптон,Нападающие
776,Морган Джиммм,17,-,-,England,Рубен Селлес,Саутгемптон,Нападающие
777,Онуачу Пауль,28,201,84,Nigeria,Рубен Селлес,Саутгемптон,Нападающие


In [ ]:
save = finalTable.to_csv('C:\\work\\JupiterNotebook\\finalTable.csv')